# nomalize (perspective warping)

In [17]:
import cv2
import math
import sys

sys.path.append("../../../")
%run ../../../ac/common/images.py
%run ../../../ac/filesystem/greps.py
%run ../../../ac/visualizer/plotter.py

%run ../../../al/actor/face.py
%run ../../../al/actor/camera.py
%run ../../../al/feature/face/candide.py

%run ../../../ds/device/nexus5x.py

# resources

In [2]:
candide_path="/home/chy/archive-model/candide/candide.npz"
land_mark_path="/home/chy/archive-model/candide/shape_predictor_68_face_landmarks.dat"
face_img_dir = "/home/chy/dev-bench/auto-catch/notes/dataset/profiles/gabe/frames"

In [3]:
face_imgs = []
for path in grep_files(face_img_dir, "*.jpg"):
    img = cv2.imread(path)
    face_imgs.append(img)
print("*** img samples {} ***".format(len(face_imgs)))

*** img samples 10 ***


# face

In [18]:
candide = Candide(candide_path, land_mark_path)
camera = MobileNexus5X.get_camera()

opt_predictor = None
face_model = FaceModel(candide)
face = Face(camera, face_model, opt_predictor, reference_fissure_length=25)

In [22]:
face.match(face_imgs[0])
face.analysis()

In [6]:
def rotate_3d_in_2d_image(img, x_degree=0, y_degree=0, z_degree=0, dx=0, dy=0, dz=0, focal_dist=1):
    cos = lambda x: math.cos(x  * math.pi / 180)
    sin = lambda x: math.sin(x  * math.pi / 180)
    
    rot_x = (x_degree - 90.) * math.pi / 180.
    rot_y = (y_degree - 90.) * math.pi / 180.
    rot_z = (z_degree - 90.) * math.pi / 180.
    
    h, w = img.shape[0], img.shape[1]
    
    # projection 2D -> 3D matrix
    proj23_mat = np.matrix([
        [1, 0, -w/2],
        [0, 1, -h/2],
        [0, 0, 1],    # [0, 0, 0]이 아니지 않나?
        [0, 0, 1]])
    
    # rotation matrix each x, y, z
    rot_x_mat = np.matrix([
        [1, 0,           0,           0],
        [0, cos(rot_x), -sin(rot_x),  0],
        [0, sin(rot_x),  cos(rot_x),  0],
        [0, 0,           0,           1]])
    
    rot_y_mat = np.matrix([
        [cos(rot_y), 0, -sin(rot_y),  0],
        [0,          1,  0,           0],
        [sin(rot_y), 0,  cos(rot_y),  0],
        [0,          0,  0,           1]])

    rot_z_mat = np.matrix([
        [cos(rot_z), -sin(rot_z),  0, 0],
        [sin(rot_z),  cos(rot_z),  0, 0],
        [0,           0,           1, 0],
        [0,           0,           0, 1]])
    
    rot_mat = rot_x_mat @ rot_y_mat @ rot_z_mat
    
    # translation matrix
    t_mat = np.matrix([
        [1, 0, 0, dx],
        [0, 1, 0, dy],
        [0, 0, 1, dz],
        [0, 0, 0, 1]])
    
    # projection 3D -> 2D matrix
    f = focal_dist
    proj32_mat = np.matrix([
        [f, 0, w/2, 0],
        [0, f, h/2, 0],
        [0, 0, 1,   0]])
    
    tran_mat = proj32_mat @ t_mat @ rot_mat @ proj23_mat
    transformed_img = cv2.warpPerspective(img, tran_mat, (w, h), flags=cv2.WARP_INVERSE_MAP)
    
    return transformed_img

In [7]:
out = rotate_3d_in_2d_image(gi, 87, 85, 90, dy=300, focal_dist=1.2)
show_image(out)

NameError: name 'gi' is not defined